In [1]:
## Calling required libraries

In [2]:
import pandas as pd
import pickle
import os

In [3]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('display.max_colwidth',None)

In [4]:
## Loading the data

In [5]:
pickle_in = open("Final DATA.pickle","rb")
Base_Data = pickle.load(pickle_in)
pickle_in.close()

In [6]:
### Getting NULL values for categorical variables

In [7]:
## Getting numeric/categorical columns

In [8]:
print(Base_Data.columns)

Num_Cols = Base_Data.select_dtypes(include='number')
char_cols = Base_Data.select_dtypes(include='string')

print(char_cols.columns)
print(Num_Cols.columns)

Index(['SRC_CLAIM_NBR', 'RECEIVED_DATE', 'PROCESSED_DATE', 'BILL_TYPE_CD',
       'REVENUE_CD_0421', 'REVENUE_CD_0551', 'REVENUE_CD_0431',
       'REVENUE_CD_0023', 'REVENUE_CD_0270', 'REVENUE_CD_0623',
       ...
       'REIMB_METH_CD_APC', 'REIMB_METH_CD_FFL', 'REIMB_METH_CD_MISSING',
       'REIMB_METH_CD_PCE', 'REIMB_METH_CD_OTHERS', 'SERV_CAT_TYPE_CD_T06',
       'SERV_CAT_TYPE_CD_A05', 'SERV_CAT_TYPE_CD_A12',
       'SERV_CAT_TYPE_CD_OTHERS', 'STATUS'],
      dtype='object', length=113)
Index(['BILL_TYPE_CD', 'ADMIT_SRC_CD', 'PRIMARY_DIAG_CD_LEVEL_1',
       'PRIMARY_DIAG_CD_LEVEL_2', 'SRC_LOB_CD', 'SRC_CAUSE_CD', 'CLIENT_NBR',
       'ADMIT_TYPE_CD', 'SERV_STATE_CD', 'STATUS'],
      dtype='object')
Index(['SRC_CLAIM_NBR', 'REVENUE_CD_0421', 'REVENUE_CD_0551',
       'REVENUE_CD_0431', 'REVENUE_CD_0023', 'REVENUE_CD_0270',
       'REVENUE_CD_0623', 'REVENUE_CD_0441', 'REVENUE_CD_0571',
       'REVENUE_CD_0561',
       ...
       'CTRCT_MOP_CD_OTHERS', 'REIMB_METH_CD_APC', 'REIMB

In [9]:
## getting number of service days in SERV_DAYS column

In [10]:
Num_Cols['PROCESSED_DAYS'] = Base_Data['PROCESSED_DATE'] - Base_Data['RECEIVED_DATE']
Num_Cols['PROCESSED_DAYS'] = Num_Cols['PROCESSED_DAYS'].astype(str)
Num_Cols['PROCESSED_DAYS'] = Num_Cols['PROCESSED_DAYS'].str[0:-4]
Num_Cols['PROCESSED_DAYS'] = Num_Cols['PROCESSED_DAYS'].apply(pd.to_numeric)
print(Num_Cols.columns)

Index(['SRC_CLAIM_NBR', 'REVENUE_CD_0421', 'REVENUE_CD_0551',
       'REVENUE_CD_0431', 'REVENUE_CD_0023', 'REVENUE_CD_0270',
       'REVENUE_CD_0623', 'REVENUE_CD_0441', 'REVENUE_CD_0571',
       'REVENUE_CD_0561',
       ...
       'REIMB_METH_CD_APC', 'REIMB_METH_CD_FFL', 'REIMB_METH_CD_MISSING',
       'REIMB_METH_CD_PCE', 'REIMB_METH_CD_OTHERS', 'SERV_CAT_TYPE_CD_T06',
       'SERV_CAT_TYPE_CD_A05', 'SERV_CAT_TYPE_CD_A12',
       'SERV_CAT_TYPE_CD_OTHERS', 'PROCESSED_DAYS'],
      dtype='object', length=102)


In [11]:
##### categorical columns stored as numeric

  #All REV , CPT and REJ (These cols are already dummified)
  #CLAIM_NBR
  
###Factor columns (combining the categorical variables in the Num_cols except the REV , CPT and REJ)

In [12]:
Fact_cols = pd.concat([char_cols,Num_Cols[['SRC_CLAIM_NBR']]], axis=1)

In [13]:
### Getting the NULL %age for each column in Fact_cols table

In [14]:
x = Fact_cols.columns
for i in x:
    print(i,":", Fact_cols[i].isnull().sum()/len(Fact_cols["SRC_CLAIM_NBR"])*100)

BILL_TYPE_CD : 0.00021327100132867834
ADMIT_SRC_CD : 0.03305700520594514
PRIMARY_DIAG_CD_LEVEL_1 : 0.8591622288525808
PRIMARY_DIAG_CD_LEVEL_2 : 0.8591622288525808
SRC_LOB_CD : 0.0
SRC_CAUSE_CD : 0.03263046320328779
CLIENT_NBR : 0.03241719220195911
ADMIT_TYPE_CD : 0.05022532081290375
SERV_STATE_CD : 0.0
STATUS : 0.0
SRC_CLAIM_NBR : 0.0


In [15]:
## There are 6 columns with missing values ( SRC_CAUSE_CD,CAS_CAUSE,SUSP,BILL_TYPE_CD,EXCode,EDIT)

#### Getting claims' distribution for various categorical variables 

####################################################

### Function to copy data on SRC_DRG_CDpboard

In [16]:
def write_to_excel(x,index=False,header=True):
    x.to_clipboard(excel=False,index=index,header=header)

In [17]:
############################
##Clubbing the minority class of Categorical variables into 'OTHERS'
############################

In [18]:
## SRC_LOB_CD

In [19]:
grp = Fact_cols.groupby('SRC_LOB_CD')
Claim_Cnt = grp['SRC_CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [20]:
##  SRC_LOB_CD replacing NULLS with MISSING factor

In [21]:
Fact_cols['SRC_LOB_CD'] = Fact_cols['SRC_LOB_CD'].fillna('MISSING')

In [22]:
###### Replacing minor level+NULLS SRC_LOB_CD with 'OTHERS'

In [23]:
SRC_LOB_CD = pd.DataFrame(Fact_cols['SRC_LOB_CD'])
SRC_LOB_CD.iloc[(SRC_LOB_CD['SRC_LOB_CD'] !='3I') & (SRC_LOB_CD['SRC_LOB_CD'] !='32') & (SRC_LOB_CD['SRC_LOB_CD'] !='39')
                     & (SRC_LOB_CD['SRC_LOB_CD'] !='62') & (SRC_LOB_CD['SRC_LOB_CD'] !='3G'), 'SRC_LOB_CD'] = 'OTHERS'

print(SRC_LOB_CD['SRC_LOB_CD'].value_counts())


3I        533247
32        293091
39         84461
62         19967
3G          6437
OTHERS       571
Name: SRC_LOB_CD, dtype: Int64


In [24]:
## SRC_CAUSE_CD

In [25]:
grp = Fact_cols.groupby('SRC_CAUSE_CD')
Claim_Cnt = grp['SRC_CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [26]:
Fact_cols['SRC_CAUSE_CD'] = Fact_cols['SRC_CAUSE_CD'].fillna('MISSING')

In [27]:
###### Replacing minor level+NULLS SRC_CAUSE_CD with 'OTHERS'

In [28]:
SRC_CAUSE_CD = pd.DataFrame(Fact_cols['SRC_CAUSE_CD'])
SRC_CAUSE_CD.iloc[(SRC_CAUSE_CD['SRC_CAUSE_CD'] != 'ILL') & (SRC_CAUSE_CD['SRC_CAUSE_CD'] != 'ACC') & (SRC_CAUSE_CD['SRC_CAUSE_CD'] != 'DIAB') & (SRC_CAUSE_CD['SRC_CAUSE_CD'] != 'COPD')
         & (SRC_CAUSE_CD['SRC_CAUSE_CD'] != 'NM') & (SRC_CAUSE_CD['SRC_CAUSE_CD'] != 'COVID'), 'SRC_CAUSE_CD'] = 'OTHERS'

print(SRC_CAUSE_CD['SRC_CAUSE_CD'].value_counts())

ILL       728685
ACC        78284
DIAB       71517
COPD       36147
NM         12330
COVID       8040
OTHERS      2771
Name: SRC_CAUSE_CD, dtype: Int64


In [29]:
## ADMIT_SRC_CD

In [30]:
grp = Fact_cols.groupby('ADMIT_SRC_CD')
Claim_Cnt = grp['SRC_CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [31]:
Fact_cols['ADMIT_SRC_CD'] = Fact_cols['ADMIT_SRC_CD'].fillna('MISSING')

In [32]:
###### Replacing minor level+NULLS ADMIT_SRC_CD with 'OTHERS'

In [33]:
ADMIT_SRC_CD = pd.DataFrame(Fact_cols['ADMIT_SRC_CD'])
ADMIT_SRC_CD.iloc[(ADMIT_SRC_CD['ADMIT_SRC_CD'] != '1') & (ADMIT_SRC_CD['ADMIT_SRC_CD'] != '4') & (ADMIT_SRC_CD['ADMIT_SRC_CD'] != '2') & 
(ADMIT_SRC_CD['ADMIT_SRC_CD'] != '5') & (ADMIT_SRC_CD['ADMIT_SRC_CD'] != '6') & (ADMIT_SRC_CD['ADMIT_SRC_CD'] != '9'), 'ADMIT_SRC_CD'] = 'OTHERS'

print(ADMIT_SRC_CD['ADMIT_SRC_CD'].value_counts())

1         410223
4         283083
2         140163
5          79159
6          15472
9           8401
OTHERS      1273
Name: ADMIT_SRC_CD, dtype: Int64


In [34]:
## ADMIT_TYPE_CD

In [35]:
grp = Fact_cols.groupby('ADMIT_TYPE_CD')
Claim_Cnt = grp['SRC_CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [36]:
Fact_cols['ADMIT_TYPE_CD'] = Fact_cols['ADMIT_TYPE_CD'].fillna('MISSING')

In [37]:
###### Replacing minor level+NULLS ADMIT_TYPE_CD with 'OTHERS'

In [38]:
ADMIT_TYPE_CD = pd.DataFrame(Fact_cols['ADMIT_TYPE_CD'])
ADMIT_TYPE_CD.iloc[(ADMIT_TYPE_CD['ADMIT_TYPE_CD'] != '9') & (ADMIT_TYPE_CD['ADMIT_TYPE_CD'] != '3') &
(ADMIT_TYPE_CD['ADMIT_TYPE_CD'] != '2'), 'ADMIT_TYPE_CD'] = 'OTHERS'

print(ADMIT_TYPE_CD['ADMIT_TYPE_CD'].value_counts())

9         713869
3         213800
2           8855
OTHERS      1250
Name: ADMIT_TYPE_CD, dtype: Int64


In [39]:
## PRIMARY_DIAG_CD_LEVEL_1

In [40]:
grp = Fact_cols.groupby('PRIMARY_DIAG_CD_LEVEL_1')
Claim_Cnt = grp['SRC_CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [41]:
Fact_cols['PRIMARY_DIAG_CD_LEVEL_1'] = Fact_cols['PRIMARY_DIAG_CD_LEVEL_1'].fillna('MISSING')

In [42]:
###### Replacing minor level+NULLS PRIMARY_DIAG_CD_LEVEL_1 with 'OTHERS'

In [45]:
PRIMARY_DIAG_CD_LEVEL_1 = pd.DataFrame(Fact_cols['PRIMARY_DIAG_CD_LEVEL_1'])
PRIMARY_DIAG_CD_LEVEL_1.iloc[(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Encounters for other specific health care') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Hypertensive diseases') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Diabetes mellitus') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Cerebrovascular diseases') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Other disorders of the skin and subcutaneous tissue') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Chronic lower respiratory diseases') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != '"Complications of surgical and medical care, not elsewhere classified"') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Other forms of heart disease') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Injuries to the hip and thigh') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Other diseases of the urinary system') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != '"Diseases of veins, lymphatic vessels and lymph nodes, not elsewhere classified"') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Osteoarthritis') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Injuries to the knee and lower leg') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Influenza and pneumonia') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Other disorders of the nervous system') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Infections of the skin and subcutaneous tissue') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Other diseases of the respiratory system') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Extrapyramidal and movement disorders') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Disorders of bone density and structure') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Other bacterial diseases') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Ischemic heart diseases') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Spondylopathies') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Other degenerative diseases of the nervous system') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Other dorsopathies') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != '"Injuries to the abdomen, lower back, lumbar spine, pelvis and external genitals"') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Mental disorders due to known physiological conditions') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Injuries to the shoulder and upper arm') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'MISSING') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Injuries to the thorax') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Injuries to the head') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Metabolic disorders') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Acute kidney failure and chronic kidney disease') &
(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'] != 'Episodic and paroxysmal disorders'), 'PRIMARY_DIAG_CD_LEVEL_1'] = 'OTHERS'

print(PRIMARY_DIAG_CD_LEVEL_1['PRIMARY_DIAG_CD_LEVEL_1'].value_counts())

Encounters for other specific health care                                            145465
OTHERS                                                                               130137
Hypertensive diseases                                                                105583
Diabetes mellitus                                                                     71670
Cerebrovascular diseases                                                              42971
Other disorders of the skin and subcutaneous tissue                                   39477
Chronic lower respiratory diseases                                                    37661
"Complications of surgical and medical care, not elsewhere classified"                29371
Other forms of heart disease                                                          26135
Injuries to the hip and thigh                                                         25559
Other diseases of the urinary system                                            

In [46]:
## PRIMARY_DIAG_CD_LEVEL_2

In [47]:
grp = Fact_cols.groupby('PRIMARY_DIAG_CD_LEVEL_2')
Claim_Cnt = grp['SRC_CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [48]:
Fact_cols['PRIMARY_DIAG_CD_LEVEL_2'] = Fact_cols['PRIMARY_DIAG_CD_LEVEL_2'].fillna('MISSING')

In [49]:
###### Replacing minor level+NULLS PRIMARY_DIAG_CD_LEVEL_2 with 'OTHERS'

In [50]:
PRIMARY_DIAG_CD_LEVEL_2 = pd.DataFrame(Fact_cols['PRIMARY_DIAG_CD_LEVEL_2'])
PRIMARY_DIAG_CD_LEVEL_2.iloc[(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != 'Diseases of the circulatory system') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != 'Factors influencing health status and contact with health services') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != '"Injury, poisoning and certain other consequences of external causes"') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != '"Endocrine, nutritional and metabolic diseases"') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != 'Diseases of the respiratory system') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != 'Diseases of the musculoskeletal system and connective tissue') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != 'Diseases of the nervous system') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != 'Diseases of the skin and subcutaneous tissue') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != 'Diseases of the genitourinary system') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != 'Neoplasms') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != 'Diseases of the digestive system') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != 'Certain infectious and parasitic diseases') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != '"Mental, Behavioral and Neurodevelopmental disorders"') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != 'MISSING') &
(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'] != 'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism'),'PRIMARY_DIAG_CD_LEVEL_2'] = 'OTHERS'

print(PRIMARY_DIAG_CD_LEVEL_2['PRIMARY_DIAG_CD_LEVEL_2'].value_counts())

Diseases of the circulatory system                                                                     220482
Factors influencing health status and contact with health services                                     145505
"Injury, poisoning and certain other consequences of external causes"                                  114366
"Endocrine, nutritional and metabolic diseases"                                                         82544
Diseases of the respiratory system                                                                      73644
Diseases of the musculoskeletal system and connective tissue                                            68229
Diseases of the nervous system                                                                          57778
Diseases of the skin and subcutaneous tissue                                                            55590
Diseases of the genitourinary system                                                                    34669
Neoplasms 

In [51]:
## CLIENT_NBR

In [52]:
grp = Fact_cols.groupby('CLIENT_NBR')
Claim_Cnt = grp['SRC_CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [53]:
Fact_cols['CLIENT_NBR'] = Fact_cols['CLIENT_NBR'].fillna('MISSING')

In [54]:
###### Replacing minor level+NULLS CLIENT_NBR with 'OTHERS'

In [55]:
CLIENT_NBR = pd.DataFrame(Fact_cols['CLIENT_NBR'])
CLIENT_NBR.iloc[(CLIENT_NBR['CLIENT_NBR'] != '58') & (CLIENT_NBR['CLIENT_NBR'] != '57') & (CLIENT_NBR['CLIENT_NBR'] != '56'), 'CLIENT_NBR'] = 'OTHERS'

print(CLIENT_NBR['CLIENT_NBR'].value_counts())

58        713574
57        134933
56         88963
OTHERS       304
Name: CLIENT_NBR, dtype: Int64


In [56]:
## SERV_STATE_CD

In [57]:
grp = Fact_cols.groupby('SERV_STATE_CD')
Claim_Cnt = grp['SRC_CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [58]:
Fact_cols['SERV_STATE_CD'] = Fact_cols['SERV_STATE_CD'].fillna('MISSING')

In [59]:
###### Replacing minor level+NULLS SERV_STATE_CD with 'OTHERS'

In [60]:
SERV_STATE_CD = pd.DataFrame(Fact_cols['SERV_STATE_CD'])
SERV_STATE_CD.iloc[
(SERV_STATE_CD['SERV_STATE_CD'] != 'IL') & (SERV_STATE_CD['SERV_STATE_CD'] != 'VA') & (SERV_STATE_CD['SERV_STATE_CD'] != 'MS') &
(SERV_STATE_CD['SERV_STATE_CD'] != 'TX') & (SERV_STATE_CD['SERV_STATE_CD'] != 'TN') & (SERV_STATE_CD['SERV_STATE_CD'] != 'AZ') &
(SERV_STATE_CD['SERV_STATE_CD'] != 'LA') & (SERV_STATE_CD['SERV_STATE_CD'] != 'MO') & (SERV_STATE_CD['SERV_STATE_CD'] != 'MN') &
(SERV_STATE_CD['SERV_STATE_CD'] != 'PA') & (SERV_STATE_CD['SERV_STATE_CD'] != 'SC') & (SERV_STATE_CD['SERV_STATE_CD'] != 'IN') &
(SERV_STATE_CD['SERV_STATE_CD'] != 'OH') & (SERV_STATE_CD['SERV_STATE_CD'] != 'NY') & (SERV_STATE_CD['SERV_STATE_CD'] != 'MD') &
(SERV_STATE_CD['SERV_STATE_CD'] != 'NC') & (SERV_STATE_CD['SERV_STATE_CD'] != 'WI') & (SERV_STATE_CD['SERV_STATE_CD'] != 'KY') &
(SERV_STATE_CD['SERV_STATE_CD'] != 'MI') & (SERV_STATE_CD['SERV_STATE_CD'] != 'WV') & (SERV_STATE_CD['SERV_STATE_CD'] != 'UT') &
(SERV_STATE_CD['SERV_STATE_CD'] != 'FL') & (SERV_STATE_CD['SERV_STATE_CD'] != 'OK') & (SERV_STATE_CD['SERV_STATE_CD'] != 'KS') &
(SERV_STATE_CD['SERV_STATE_CD'] != 'CA') & (SERV_STATE_CD['SERV_STATE_CD'] != 'WA') & (SERV_STATE_CD['SERV_STATE_CD'] != 'AL') &
(SERV_STATE_CD['SERV_STATE_CD'] != 'GA'), 'SERV_STATE_CD'] = 'OTHERS'

print(SERV_STATE_CD['SERV_STATE_CD'].value_counts())

IL        210525
TX        179639
LA         82498
PA         56536
OH         47674
OTHERS     37993
NC         37152
MI         36023
FL         34255
CA         33752
GA         21035
VA         15923
TN         15117
MO         13851
SC         13781
NY         13383
WI          8441
WV          8263
OK          8133
WA          7878
MS          7700
AZ          7469
MN          7283
KY          7014
UT          6848
IN          6700
KS          6197
AL          5522
MD          1189
Name: SERV_STATE_CD, dtype: Int64


In [61]:
## BILL_TYPE_CD

In [62]:
grp = Fact_cols.groupby('BILL_TYPE_CD')
Claim_Cnt = grp['SRC_CLAIM_NBR'].nunique()

write_to_excel(Claim_Cnt)

In [63]:
Fact_cols['BILL_TYPE_CD'] = Fact_cols['BILL_TYPE_CD'].fillna('MISSING')

In [64]:
###### Replacing minor level+NULLS BILL_TYPE_CD with 'OTHERS'

In [65]:
BILL_TYPE_CD = pd.DataFrame(Fact_cols['BILL_TYPE_CD'])
BILL_TYPE_CD.iloc[(BILL_TYPE_CD['BILL_TYPE_CD'] != '329') & (BILL_TYPE_CD['BILL_TYPE_CD'] != '323') & (BILL_TYPE_CD['BILL_TYPE_CD'] != '322') &
(BILL_TYPE_CD['BILL_TYPE_CD'] != '324') & (BILL_TYPE_CD['BILL_TYPE_CD'] != '321') & (BILL_TYPE_CD['BILL_TYPE_CD'] != '327'), 'BILL_TYPE_CD'] = 'OTHERS'

print(BILL_TYPE_CD['BILL_TYPE_CD'].value_counts())

329       730660
323       100514
322        49179
324        22833
321        22756
327         9654
OTHERS      2178
Name: BILL_TYPE_CD, dtype: Int64


In [66]:
### Cbind the above categorical variable

In [67]:
Comb_Fact_Data = pd.DataFrame(pd.concat([ADMIT_SRC_CD,PRIMARY_DIAG_CD_LEVEL_1,PRIMARY_DIAG_CD_LEVEL_2,SRC_LOB_CD,SRC_CAUSE_CD,CLIENT_NBR,ADMIT_TYPE_CD,SERV_STATE_CD,BILL_TYPE_CD],axis=1))
print(Comb_Fact_Data.columns)

Index(['ADMIT_SRC_CD', 'PRIMARY_DIAG_CD_LEVEL_1', 'PRIMARY_DIAG_CD_LEVEL_2',
       'SRC_LOB_CD', 'SRC_CAUSE_CD', 'CLIENT_NBR', 'ADMIT_TYPE_CD',
       'SERV_STATE_CD', 'BILL_TYPE_CD'],
      dtype='object')


In [68]:
######## Removing the above cols from the Fact_cols data

In [69]:
Fact_cols = Fact_cols.drop(['ADMIT_SRC_CD','PRIMARY_DIAG_CD_LEVEL_1','PRIMARY_DIAG_CD_LEVEL_2','SRC_LOB_CD','SRC_CAUSE_CD','CLIENT_NBR','ADMIT_TYPE_CD','SERV_STATE_CD','BILL_TYPE_CD'],axis=1)

In [70]:
##### Adding the revised dataset to the Fact_cols table

In [71]:
Fact_cols = pd.concat([Fact_cols, Comb_Fact_Data],axis=1)

In [72]:
del ADMIT_SRC_CD,PRIMARY_DIAG_CD_LEVEL_1,PRIMARY_DIAG_CD_LEVEL_2,SRC_LOB_CD,SRC_CAUSE_CD,CLIENT_NBR,ADMIT_TYPE_CD,SERV_STATE_CD,BILL_TYPE_CD, x

In [73]:
## converting dtype of categorical columns

In [74]:
Fact_cols = Fact_cols.astype({'ADMIT_SRC_CD':'category','PRIMARY_DIAG_CD_LEVEL_1':'category','PRIMARY_DIAG_CD_LEVEL_2':'category','SRC_LOB_CD':'category','SRC_CAUSE_CD':'category','CLIENT_NBR':'category','ADMIT_TYPE_CD':'category','SERV_STATE_CD':'category','BILL_TYPE_CD':'category','STATUS':'category'})

In [75]:
## changing dtype of numerical columns

In [76]:
print(Num_Cols.columns)

Index(['SRC_CLAIM_NBR', 'REVENUE_CD_0421', 'REVENUE_CD_0551',
       'REVENUE_CD_0431', 'REVENUE_CD_0023', 'REVENUE_CD_0270',
       'REVENUE_CD_0623', 'REVENUE_CD_0441', 'REVENUE_CD_0571',
       'REVENUE_CD_0561',
       ...
       'REIMB_METH_CD_APC', 'REIMB_METH_CD_FFL', 'REIMB_METH_CD_MISSING',
       'REIMB_METH_CD_PCE', 'REIMB_METH_CD_OTHERS', 'SERV_CAT_TYPE_CD_T06',
       'SERV_CAT_TYPE_CD_A05', 'SERV_CAT_TYPE_CD_A12',
       'SERV_CAT_TYPE_CD_OTHERS', 'PROCESSED_DAYS'],
      dtype='object', length=102)


In [77]:
Num_Cols.dtypes

SRC_CLAIM_NBR                 Int64
REVENUE_CD_0421               Int64
REVENUE_CD_0551               Int64
REVENUE_CD_0431               Int64
REVENUE_CD_0023               Int64
REVENUE_CD_0270               Int64
REVENUE_CD_0623               Int64
REVENUE_CD_0441               Int64
REVENUE_CD_0571               Int64
REVENUE_CD_0561               Int64
REVENUE_CD_0550               Int64
REVENUE_CD_0420               Int64
REVENUE_CD_OTHERS             Int64
SERV_UNIT_CNT               Float64
NET_PAID_AMT                Float64
DEDUCT_AMT                  Float64
MBR_COPAY_AMT               Float64
MBR_RESP_AMT                Float64
CPT_MAPPING_1                 Int64
CPT_MAPPING_2                 Int64
CPT_MAPPING_3                 Int64
CPT_MAPPING_4                 Int64
CPT_MAPPING_OTHERS            Int64
HIPPS_MAPPING_1               Int64
HIPPS_MAPPING_2               Int64
HIPPS_MAPPING_3               Int64
HIPPS_MAPPING_OTHERS          Int64
OVERRIDE_CD_27              

In [81]:
Num_Cols = Num_Cols.astype({
'REVENUE_CD_0421'               :'category',
'REVENUE_CD_0551'               :'category',
'REVENUE_CD_0431'               :'category',
'REVENUE_CD_0023'               :'category',
'REVENUE_CD_0270'               :'category',
'REVENUE_CD_0623'               :'category',
'REVENUE_CD_0441'               :'category',
'REVENUE_CD_0571'               :'category',
'REVENUE_CD_0561'               :'category',
'REVENUE_CD_0550'               :'category',
'REVENUE_CD_0420'               :'category',
'REVENUE_CD_OTHERS'             :'category',
'CPT_MAPPING_1'                 :'category',
'CPT_MAPPING_2'                 :'category',
'CPT_MAPPING_3'                 :'category',
'CPT_MAPPING_4'                 :'category',
'CPT_MAPPING_OTHERS'            :'category',
'HIPPS_MAPPING_1'               :'category',
'HIPPS_MAPPING_2'               :'category',
'HIPPS_MAPPING_3'               :'category',
'HIPPS_MAPPING_OTHERS'          :'category',
'OVERRIDE_CD_27'                :'category',
'OVERRIDE_CD_00'                :'category',
'OVERRIDE_CD_34'                :'category',
'OVERRIDE_CD_37'                :'category',
'OVERRIDE_CD_OTHERS'           :'category',
'CTRCT_CAT_CD_01'               :'category',
'CTRCT_CAT_CD_04'               :'category',
'CTRCT_CAT_CD_OTHERS'           :'category',
'SRC_SERV_TYPE_CD1_421'         :'category',
'SRC_SERV_TYPE_CD1_551'         :'category',
'SRC_SERV_TYPE_CD1_431'         :'category',
'SRC_SERV_TYPE_CD1_023'         :'category',
'SRC_SERV_TYPE_CD1_270'         :'category',
'SRC_SERV_TYPE_CD1_623'         :'category',
'SRC_SERV_TYPE_CD1_441'         :'category',
'SRC_SERV_TYPE_CD1_571'         :'category',
'SRC_SERV_TYPE_CD1_561'         :'category',
'SRC_SERV_TYPE_CD1_550'         :'category',
'SRC_SERV_TYPE_CD1_420'         :'category',
'SRC_SERV_TYPE_CD1_OTHERS'      :'category',
'SRC_SERV_TYPE_CD2_Q5001'       :'category',
'SRC_SERV_TYPE_CD2_G0151'       :'category',
'SRC_SERV_TYPE_CD2_G0299'       :'category',
'SRC_SERV_TYPE_CD2_G0152'       :'category',
'SRC_SERV_TYPE_CD2_G0157'       :'category',
'SRC_SERV_TYPE_CD2_G0300'       :'category',
'SRC_SERV_TYPE_CD2_G0158'       :'category',
'SRC_SERV_TYPE_CD2_G0153'       :'category',
'SRC_SERV_TYPE_CD2_G0156'       :'category',
'SRC_SERV_TYPE_CD2_Q5002'       :'category',
'SRC_SERV_TYPE_CD2_G0155'       :'category',
'SRC_SERV_TYPE_CD2_G0495'       :'category',
'SRC_SERV_TYPE_CD2_OTHERS'      :'category',
'PAYEE_CD_5'                    :'category',
'PAYEE_CD_9'                    :'category',
'PAYEE_CD_I'                    :'category',
'PAYEE_CD_OTHERS'               :'category',
'REASON_CD1_8A0DC'              :'category',
'REASON_CD1_8A21Q'              :'category',
'REASON_CD1_8A29T'              :'category',
'REASON_CD1_MISSING'            :'category',
'REASON_CD1_7A0DC'              :'category',
'REASON_CD1_7A21Q'              :'category',
'REASON_CD1_6A0DC'              :'category',
'REASON_CD1_6A21Q'              :'category',
'REASON_CD1_8A46R'              :'category',
'REASON_CD1_8A52Q'              :'category',
'REASON_CD1_8_21Q'              :'category',
'REASON_CD1_7A29T'              :'category',
'REASON_CD1_8A04'               :'category',
'REASON_CD1_7A04'               :'category',
'REASON_CD1_OTHERS'             :'category',
'PROV_TYPE_CD_HH'               :'category',
'PROV_TYPE_CD_VENDOR'           :'category',
'PROV_TYPE_CD_RN'               :'category',
'PROV_TYPE_CD_MD'               :'category',
'PROV_TYPE_CD_SS'               :'category',
'PROV_TYPE_CD_OTHERS'           :'category',
'CTRCT_MOP_CD_PS'               :'category',
'CTRCT_MOP_CD_PV'               :'category',
'CTRCT_MOP_CD_PC'               :'category',
'CTRCT_MOP_CD_MISSING'          :'category',
'CTRCT_MOP_CD_OTHERS'           :'category',
'REIMB_METH_CD_APC'             :'category',
'REIMB_METH_CD_FFL'             :'category',
'REIMB_METH_CD_MISSING'         :'category',
'REIMB_METH_CD_PCE'             :'category',
'REIMB_METH_CD_OTHERS'          :'category',
'SERV_CAT_TYPE_CD_T06'          :'category',
'SERV_CAT_TYPE_CD_A05'          :'category',
'SERV_CAT_TYPE_CD_A12'          :'category',
'SERV_CAT_TYPE_CD_OTHERS'       :'category'
})

In [82]:
## Combining Categorical and Numerical columns

In [83]:
Num_Cols = Num_Cols.drop(['SRC_CLAIM_NBR'],axis=1)

In [84]:
Final_Data = pd.concat([Fact_cols,Num_Cols], axis=1)
Final_Data.index = Fact_cols.index
#Final_Data = Final_Data.drop(['CLAIM_NBR'],axis=1)

In [85]:
x = Final_Data.columns
for i in x:
    print(i,":", Final_Data[i].isnull().sum()/len(Fact_cols["SRC_CLAIM_NBR"])*100)

STATUS : 0.0
SRC_CLAIM_NBR : 0.0
ADMIT_SRC_CD : 0.0
PRIMARY_DIAG_CD_LEVEL_1 : 0.0
PRIMARY_DIAG_CD_LEVEL_2 : 0.0
SRC_LOB_CD : 0.0
SRC_CAUSE_CD : 0.0
CLIENT_NBR : 0.0
ADMIT_TYPE_CD : 0.0
SERV_STATE_CD : 0.0
BILL_TYPE_CD : 0.0
REVENUE_CD_0421 : 0.0
REVENUE_CD_0551 : 0.0
REVENUE_CD_0431 : 0.0
REVENUE_CD_0023 : 0.0
REVENUE_CD_0270 : 0.0
REVENUE_CD_0623 : 0.0
REVENUE_CD_0441 : 0.0
REVENUE_CD_0571 : 0.0
REVENUE_CD_0561 : 0.0
REVENUE_CD_0550 : 0.0
REVENUE_CD_0420 : 0.0
REVENUE_CD_OTHERS : 0.0
SERV_UNIT_CNT : 0.0
NET_PAID_AMT : 0.0
DEDUCT_AMT : 0.0
MBR_COPAY_AMT : 0.0
MBR_RESP_AMT : 0.0
CPT_MAPPING_1 : 0.0
CPT_MAPPING_2 : 0.0
CPT_MAPPING_3 : 0.0
CPT_MAPPING_4 : 0.0
CPT_MAPPING_OTHERS : 0.0
HIPPS_MAPPING_1 : 0.0
HIPPS_MAPPING_2 : 0.0
HIPPS_MAPPING_3 : 0.0
HIPPS_MAPPING_OTHERS : 0.0
OVERRIDE_CD_27 : 0.0
OVERRIDE_CD_00 : 0.0
OVERRIDE_CD_34 : 0.0
OVERRIDE_CD_37 : 0.0
OVERRIDE_CD_OTHERS : 0.0
CTRCT_CAT_CD_01 : 0.0
CTRCT_CAT_CD_04 : 0.0
CTRCT_CAT_CD_OTHERS : 0.0
MBR_AGE : 0.0
SRC_SERV_TYPE_CD1_421 : 

In [86]:
pickle_out = open('Final_Data_.pickle','wb')
pickle.dump(Final_Data,pickle_out)
pickle_out.close()

In [87]:
Final_Data.dtypes

STATUS                      category
SRC_CLAIM_NBR                  Int64
ADMIT_SRC_CD                category
PRIMARY_DIAG_CD_LEVEL_1     category
PRIMARY_DIAG_CD_LEVEL_2     category
SRC_LOB_CD                  category
SRC_CAUSE_CD                category
CLIENT_NBR                  category
ADMIT_TYPE_CD               category
SERV_STATE_CD               category
BILL_TYPE_CD                category
REVENUE_CD_0421             category
REVENUE_CD_0551             category
REVENUE_CD_0431             category
REVENUE_CD_0023             category
REVENUE_CD_0270             category
REVENUE_CD_0623             category
REVENUE_CD_0441             category
REVENUE_CD_0571             category
REVENUE_CD_0561             category
REVENUE_CD_0550             category
REVENUE_CD_0420             category
REVENUE_CD_OTHERS           category
SERV_UNIT_CNT                Float64
NET_PAID_AMT                 Float64
DEDUCT_AMT                   Float64
MBR_COPAY_AMT                Float64
M